In [1]:
from bs4 import BeautifulSoup
from pyquery import PyQuery as pq
from collections import defaultdict 
from imdb import IMDb
import cPickle as pickle
import io 
import time
import requests

### Scrape the movie gross Information from boxofficemojo.com 

In [20]:
years = xrange(2009,2015)
pages = xrange(1,9)
year_pagetxt = {}
for year in years: 
    pagestext = {}
    for page in pages: 
        r = requests.get("http://www.boxofficemojo.com/yearly/chart/?page=%s&view=releasedate&view2=domestic&yr=%s&p=.htm"%(page, year))
        pagestext[page] = r.text
        time.sleep(1)
    year_pagetxt[year] = pagestext

In [21]:
movie_budget = defaultdict(list) 
for year in years: 
    for page in pages: 
        soup = BeautifulSoup(year_pagetxt[year][page], "html.parser")
        rows = soup.find_all("font", attrs={'size':'2'})
           
        start = 10 
        for i in range(start,len(rows)-2):
            t = rows[i].get_text()
            if unicode('Summary of') in t: 
                break
            elif (i-start) % 9 == 0: 
                movie_budget['rank'].append(t)
            elif (i-start) % 9 == 1: 
                movie_budget['year'].append(year)
                r = '('+str(year)
                if unicode(r) in t: 
                    j = t.index(unicode(r))
                    movie_budget['title'].append(t[:j])
                else: 
                    movie_budget['title'].append(t)
            elif (i-start) % 9 == 2: 
                movie_budget['studio'].append(t)
            elif (i-start) % 9 == 3: 
                movie_budget['gross'].append(t)
            elif (i-start) % 9 == 4: 
                movie_budget['gross theaters'].append(t)
            elif (i-start) % 9 == 5: 
                movie_budget['opening'].append(t)
            elif (i-start) % 9 == 6: 
                movie_budget['opening theaters'].append(t)
            elif (i-start) % 9 == 7: 
                movie_budget['open'].append(t)
            elif (i-start) % 9 == 8: 
                movie_budget['close'].append(t)

In [22]:
print len(movie_budget['title'])
print set(movie_budget['year'])

3715
set([2009, 2010, 2011, 2012, 2013, 2014])


###Scrape movie budget information from the-numbers.com 

In [23]:
r_numbers = requests.get("http://www.the-numbers.com/movie/budgets/all")

d_=pq(r_numbers.text)
d_tables=pq(d_('table'))
rows = pq(d_tables[0])('tr')

In [24]:
budget = defaultdict(list)
for j in range(1,len(rows)):
    dat = pq(rows[j])('td')
    for i in range(len(dat)):
        if i % 6 == 1:
            budget['year'].append(int(pq(dat[i])('a').text().split('/')[-1]))
        elif i % 6 == 2:
            t = pq(dat[i])('a').text()
            if 'Birdman' in t:
                budget['title'].append(t.split(' or ')[0])
            else:
                budget['title'].append(t)
        elif i % 6 == 3: 
            budget['budget'].append(pq(dat[i]).text())

### Search_movies might return many movies related to our search. This finds the appropriate movie.

In [25]:
def find_movie(title, year,  mlist):
    # find movies that came out in the same year                                                                                                                                    
    year_list = []
    for movie in mlist:
        try:
            if movie.data['year'] == int(year):
                year_list.append(movie)
        except:
            continue
    # if the years do not match, there is no match                                                                                                                                  
    if len(year_list) < 1:
        return None
    else:
        # process the title                                                                                                                                                         
        sorted_title = "".join(sorted(title)).replace(" ", "")
        len_sorted_title = len(sorted_title)
        # check whether movies that came out in the same year                                                                                                                       
        # have the same letters                                                                                                                                                     
        counts = [0]*len(year_list)
        for j in range(len(year_list)):
            mtitle = year_list[j]['title']
            #import ipdb;ipdb.set_trace()
            sorted_mtitle = "".join(sorted(mtitle)).replace(" ", "").replace('(I)','')
            if len_sorted_title == len(sorted_mtitle):
                # if the title cannot be converted to a string                                                                                                                      
                # it is not the correct title                                                                                                                                       
                try:
                    sorted_mtitle = str(sorted_mtitle)
                except:
                    continue
                for i in range(len_sorted_title):
                    if sorted_title[i].lower() == sorted_mtitle[i].lower():
                        counts[j] += 1
            else:
                continue
        
        if max(counts) <= len_sorted_title: 
            k = counts.index(max(counts))
        else: 
            k = counts.index(len_sorted_title)
        if len(year_list) >= 1:
            return year_list[k]
        else:
            return None

### Combine movie ids, budget info, and gross info into one dict

In [29]:
ia = IMDb(accessSystem='http')

#The next process actually attempts to match the movies in the Box Office dataframe to the 
BOmissingmovies = [] # To hold anything we don't find
BOdict = {} # This will actually contain the movies and information about them.

movienumber = len(movie_budget['gross'])
print movienumber

3715


In [30]:
for i in range(movienumber):
    movieobj = None
    # Need to process the row-level information out of BOdf in order to get the movie objects
    movieobj = ia.search_movie(movie_budget['title'][i])
    
    #Handling cases where we haven't found the movie or have multiples
    if movieobj is None or len(movieobj)>1:
        potential_movie_titles = ia.search_movie(movie_budget['title'][i])
        movieobj = find_movie(movie_budget['title'][i], movie_budget['year'][i], potential_movie_titles) # find the movie
        if type(movieobj) == list: 
            movieobj = movieobj[0]
            
    print (i, movie_budget['title'][i], movie_budget['year'][i], movieobj)
    
    if movieobj is not None and not (type(movieobj) == list and len(movieobj) == 0):
        ## Get movie id ##
        if type(movieobj) == list: 
            movieobj = movieobj[0]
        ia.update(movieobj)    
        movid = movieobj.movieID
        ## Populate dictionary, main key is movie id ##
        BOdict[movid] = {}
        # "title": title of movie
        BOdict[movid]['title'] = movie_budget['title'][i]
        BOdict[movid]['gross'] = movie_budget['gross'][i]
        BOdict[movid]['opening'] = movie_budget['opening'][i]
        BOdict[movid]['rank'] = movie_budget['rank'][i]
        BOdict[movid]['studio'] = movie_budget['studio'][i] 
        BOdict[movid]['open'] = movie_budget['open'][i]
        BOdict[movid]['close'] = movie_budget['close'][i]
        BOdict[movid]['opening theaters'] = movie_budget['opening theaters'][i]
        try:
            BOdict[movid]['rating'] = movieobj['rating']
        except: 
            BOdict[movid]['rating'] = []
        try:
            BOdict[movid]['mpaa'] = movieobj['mpaa']
        except: 
            BOdict[movid]['mpaa'] = []
        try:
            BOdict[movid]['director'] = movieobj['director']
        except: 
            BOdict[movid]['director'] = []
        try:
            BOdict[movid]['cast'] = movieobj['cast']
        except:
            BOdict[movid]['cast'] = []
        #BOdict[movid]['']
        # make each award individual key and the value to indicate whether movie won/nominated or not
        # Loop through awards list and indicate if movie was nominated or won
    else:
        BOmissingmovies.append((i, movie_budget['title'][i], movie_budget['year'][i]))
        

(0, u'Avatar', 2009, <Movie id:0499549[http] title:_Avatar (2009)_>)
(1, u'Transformers: Revenge of the Fallen', 2009, <Movie id:1055369[http] title:_Transformers: Revenge of the Fallen (2009)_>)
(2, u'Harry Potter and the Half-Blood Prince', 2009, <Movie id:0417741[http] title:_Harry Potter and the Half-Blood Prince (2009)_>)
(3, u'The Twilight Saga: New Moon', 2009, <Movie id:1259571[http] title:_The Twilight Saga: New Moon (2009)_>)
(4, u'Up', 2009, <Movie id:1049413[http] title:_Up (2009)_>)
(5, u'The Hangover', 2009, <Movie id:1119646[http] title:_The Hangover (2009)_>)
(6, u'Star Trek', 2009, <Movie id:0796366[http] title:_Star Trek (2009)_>)
(7, u'The Blind Side', 2009, <Movie id:0878804[http] title:_The Blind Side (2009)_>)
(8, u'Alvin and the Chipmunks: The Squeakquel', 2009, <Movie id:1231580[http] title:_Alvin and the Chipmunks: The Squeakquel (2009)_>)
(9, u'Sherlock Holmes', 2009, <Movie id:0988045[http] title:_Sherlock Holmes (2009)_>)
(10, u'Monsters Vs. Aliens', 2009, <

2015-12-03 23:09:38,487 CRITICAL [imdbpy] /Users/dhanus/anaconda/lib/python2.7/site-packages/imdb/_exceptions.py:35: IMDbDataAccessError exception raised; args: ({'exception type': 'IOError', 'url': 'http://akas.imdb.com/title/tt3645948/combined', 'errcode': 'socket error', 'proxy': '', 'original exception': IOError('socket error', timeout('timed out',)), 'errmsg': 'timed out'},); kwds: {}
Traceback (most recent call last):
  File "/Users/dhanus/anaconda/lib/python2.7/site-packages/imdb/parser/http/__init__.py", line 202, in retrieve_unicode
    uopener = self.open(url)
  File "/Users/dhanus/anaconda/lib/python2.7/urllib.py", line 213, in open
    return getattr(self, name)(url)
  File "/Users/dhanus/anaconda/lib/python2.7/urllib.py", line 351, in open_http
    errcode, errmsg, headers = h.getreply()
  File "/Users/dhanus/anaconda/lib/python2.7/httplib.py", line 1207, in getreply
    response = self._conn.getresponse()
  File "/Users/dhanus/anaconda/lib/python2.7/httplib.py", line 1132


(3113, u'Top Five', 2014, <Movie id:2784678[http] title:_Top Five (2014)_>)
(3114, u'The November Man', 2014, <Movie id:2402157[http] title:_The November Man (2014)_>)
(3115, u'Endless Love ', 2014, <Movie id:2318092[http] title:_Endless Love (2014)_>)
(3116, u'Pompeii', 2014, <Movie id:3507574[http] title:_"Reel Junkie" Pompeii (2014)_>)
(3117, u'Transcendence', 2014, <Movie id:3660986[http] title:_"Nook's Nifty Nitpicks" Transcendence (2014)_>)
(3118, u'As Above/So Below', 2014, <Movie id:2870612[http] title:_As Above, So Below (2014)_>)
(3119, u'Brick Mansions', 2014, <Movie id:1430612[http] title:_Brick Mansions (2014)_>)
(3120, u'I, Frankenstein', 2014, <Movie id:1418377[http] title:_I, Frankenstein (2014)_>)
(3121, u'The Legend of Hercules', 2014, <Movie id:1043726[http] title:_The Legend of Hercules (2014)_>)
(3122, u'Bears', 2014, <Movie id:2458776[http] title:_Bears (I) (2014)_>)
(3123, u'Addicted', 2014, <Movie id:2205401[http] title:_Addicted (I) (2014)_>)
(3124, u'A Haunte

In [6]:
#pickle.dump(BOdict, io.open('BOdict.p', 'wb'))
BOdict = pickle.load(io.open('BOdict.p', 'rb'))

In [3]:
# add seasons 
for k in BOdict.keys():
    open_date = BOdict[k]['open']
    if open_date:
        month = int(open_date.split('/')[0])
        if month <= 5 and month >= 1:
            BOdict[k]['season'] = 0
        elif month <= 9 and month >= 6:
            BOdict[k]['season'] = 1 
        elif month <= 11 and month >= 10:
            BOdict[k]['season'] = 2
        elif month == 12: 
            BOdict[k]['season'] = 3
        else:
            BOdict[k]['season'] = None 

In [18]:
print type(BOdict['3804080'])

<type 'dict'>


In [52]:
for movie in BOdict.keys():
    t = BOdict[movie]['title']
    try:
        j  = budget['title'].index(t)
        BOdict[movie]['budget'] = budget['budget'][j]
    except:
        continue

In [5]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")
from PIL import Image
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix

In [51]:
def cv_optimize(regmodel, parameters, X, y, n_jobs=1, n_folds=5, score_func=None):
    if score_func:
        gs = GridSearchCV(regmodel, param_grid=parameters, cv=n_folds, n_jobs=n_jobs, scoring=mean_squared_error)
    else:
        gs = GridSearchCV(regmodel, param_grid=parameters, n_jobs=n_jobs, cv=n_folds)
    gs.fit(X, y)
    print "BEST", gs.best_params_, gs.best_score_, gs.grid_scores_
    best = gs.best_estimator_
    return best

In [65]:
def do_regression(regmodel, parameters, indf, featurenames, mask=None, 
                  reuse_split=None, score_func=None, n_folds=5, n_jobs=1):
    subdf=indf[featurenames]
    X=subdf.values
    y=indf['claim_amount']
    print "where"
    if mask !=None:
        print "using mask"
        Xtrain, Xtest, ytrain, ytest = X[mask], X[~mask], y[mask], y[~mask]
    if reuse_split !=None:
        print "using reuse split"
        Xtrain, Xtest, ytrain, ytest = reuse_split['Xtrain'], reuse_split['Xtest'], reuse_split['ytrain'], reuse_split['ytest']
    if parameters:
        regmodel = cv_optimize(regmodel, parameters, Xtrain, ytrain, n_jobs=n_jobs, n_folds=n_folds, score_func=score_func)
    regmodel=regmodel.fit(Xtrain, ytrain)
    training_accuracy = regmodel.score(Xtrain, ytrain)
    test_accuracy = regmodel.score(Xtest, ytest)
    print "############# based on standard predict ################"
    print "Accuracy on training data: %0.2f" % (training_accuracy)
    print "Accuracy on test data:     %0.2f" % (test_accuracy)
    print confusion_matrix(ytest, regmodel.predict(Xtest))
    print "########################################################"
    return regmodel, Xtrain, ytrain, Xtest, ytest

In [25]:
BOdict

{'3804080': {'cast': [<Person id:6580065[http] name:_AbbaKiss_>,
   <Person id:5265152[http] name:_Corten, Anthony_>,
   <Person id:4103436[http] name:_Dellamorto, Dixie_>,
   <Person id:1495195[http] name:_Lobo, Mister_>,
   <Person id:4394074[http] name:_Milliken, Jamie_>,
   <Person id:6580066[http] name:_Mittens, Miss_>,
   <Person id:6580063[http] name:_Sakura, George_>,
   <Person id:6580064[http] name:_Something, Richard_>],
  'close': u'7/20',
  'director': [<Person id:6580067[http] name:_Sakellaropoulos, Andrea_>],
  'gross': u'$515,005',
  'mpaa': [],
  'open': u'7/4',
  'opening': u'$220,542',
  'opening theaters': u'102',
  'rank': u'252',
  'rating': [],
  'studio': u'Jan.',
  'title': u"A Hard Day's Night "},
 '1687875': {'cast': [],
  'close': u'-',
  'director': [<Person id:3979458[http] name:_Freilich, Toby Perl_>],
  'gross': u'$16,734',
  'mpaa': [],
  'open': u'4/27',
  'opening': u'$6,193',
  'opening theaters': u'1',
  'rank': u'513',
  'rating': 5.7,
  'studio': 

In [34]:
ax = {'a':{'b':1, 'c':2}}

In [39]:
pd.DataFrame.from_dict(ax, orient='index')

,c,b
a,2,1


In [46]:
sampledata=pd.read_csv('insurance_claims.csv')

In [47]:
sampledata

,claim_type,claim_amount,fraudulent,coverage,deductible,townsize,gender,edcat,job_start_date,retire,income,marital
0,Fire/Smoke,192.28700,No,436,2000,"> 250,000",Female,High school degree,4/16/81,No,193,Unmarried
1,Fire/Smoke,355.89600,No,925,1000,"50,000-249,999",Male,High school degree,11/4/82,No,203,Unmarried
2,Theft/Vandalism,3.53000,No,79,1000,"> 250,000",Female,High school degree,8/20/04,No,49,Unmarried
3,Wind/Hail,33.45200,No,607,1000,"> 250,000",Male,Some college,7/29/99,No,118,Married
4,Theft/Vandalism,4.02900,No,119,3000,"< 2,500",Female,Did not complete high school,7/4/04,No,18,Unmarried
5,Contamination,66.35800,No,144,500,"50,000-249,999",Male,Some college,6/19/69,Yes,25,Married
6,Wind/Hail,29.39100,No,636,1000,"2,500-9,999",Male,College degree,11/27/08,No,36,Unmarried
7,Water damage,28.99300,No,254,2000,"> 250,000",Female,Did not complete high school,7/24/88,Yes,13,Married
8,Contamination,128.31500,No,291,1000,"50,000-249,999",Male,College degree,9/17/97,No,91,Unmarried
9,Theft/Vandalism,4.84300,No,114,1000,"< 2,500",Female,Did not complete high school,4/11/05,No,23,Married


In [53]:
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix
itrain, itest = train_test_split(xrange(sampledata.shape[0]), train_size=0.6)
mask=np.ones(sampledata.shape[0], dtype='int')
mask[itrain]=1
mask[itest]=0
mask = (mask==1)
mask[:10]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,  True], dtype=bool)

In [67]:
from sklearn.linear_model import LinearRegression
from sklearn import metrics
regmodel = LinearRegression()
parameters = {"fit_intercept": [True, False], "normalize":[True, False], "copy_X":[True, False] }
regmodel, Xtrain, ytrain, Xtest, ytest=do_regression(regmodel, parameters, sampledata, ['coverage','deductible', 'income'], mask=mask, score_func = 'mean_squared_error')

where
using mask
BEST {'copy_X': True, 'normalize': True, 'fit_intercept': False} -15717.4044363 [mean: -15725.16197, std: 3138.38806, params: {'copy_X': True, 'normalize': True, 'fit_intercept': True}, mean: -15725.16197, std: 3138.38806, params: {'copy_X': True, 'normalize': False, 'fit_intercept': True}, mean: -15717.40444, std: 3150.30479, params: {'copy_X': True, 'normalize': True, 'fit_intercept': False}, mean: -15717.40444, std: 3150.30479, params: {'copy_X': True, 'normalize': False, 'fit_intercept': False}, mean: -15725.16197, std: 3138.38806, params: {'copy_X': False, 'normalize': True, 'fit_intercept': True}, mean: -15725.16197, std: 3138.38806, params: {'copy_X': False, 'normalize': False, 'fit_intercept': True}, mean: -15717.40444, std: 3150.30479, params: {'copy_X': False, 'normalize': True, 'fit_intercept': False}, mean: -15717.40444, std: 3150.30479, params: {'copy_X': False, 'normalize': False, 'fit_intercept': False}]
############# based on standard predict ##########

/Users/rohanmazumdar/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:7: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


ValueError: continuous is not supported

In [84]:
from sklearn.feature_selection import RFE
regmodel = LinearRegression()
rfe=RFE(regmodel)
regmodel.fit(sampledata[['coverage', 'deductible','income']][mask],sampledata['claim_amount'][mask])
training_accuracy = regmodel.score(sampledata[['coverage', 'deductible','income']][mask], sampledata['claim_amount'][mask])
test_accuracy = regmodel.score(sampledata[['coverage', 'deductible','income']][~mask], sampledata['claim_amount'][~mask])
print training_accuracy
print test_accuracy

0.260491373596
0.265995074021


In [85]:
import statsmodels.formula.api as smf

def forward_selected(data, response):
    """Linear model designed by forward selection.

    Parameters:
    -----------
    data : pandas DataFrame with all possible predictors and response

    response: string, name of response column in data

    Returns:
    --------
    model: an "optimal" fitted statsmodels linear model
           with an intercept
           selected by forward selection
           evaluated by adjusted R-squared
    """
    remaining = set(data.columns)
    remaining.remove(response)
    selected = []
    current_score, best_new_score = 0.0, 0.0
    while remaining and current_score == best_new_score:
        scores_with_candidates = []
        for candidate in remaining:
            formula = "{} ~ {} + 1".format(response,
                                           ' + '.join(selected + [candidate]))
            score = smf.ols(formula, data).fit().rsquared_adj
            scores_with_candidates.append((score, candidate))
        scores_with_candidates.sort()
        best_new_score, best_candidate = scores_with_candidates.pop()
        if current_score < best_new_score:
            remaining.remove(best_candidate)
            selected.append(best_candidate)
            current_score = best_new_score
    formula = "{} ~ {} + 1".format(response,
                                   ' + '.join(selected))
    model = smf.ols(formula, data).fit()
    return model

In [86]:
trialmodel = forward_selected(sampledata, "claim_amount")

KeyboardInterrupt: 

In [ ]:
trialmodel.summary()